======================== Import Packages ==========================

In [ ]:
import sys, pdb, glob
import numpy as np
from astropy.table import Table, join
from astroquery.vizier import Vizier
import warnings
from astropy.logger import AstropyWarning
warnings.filterwarnings('ignore', category=AstropyWarning)

===================== Define Functions ===================

In [ ]:
def get_data(catalog, join_key='Name'):

    """
    PURPOSE:    Get data from literature with Vizier

    INPUT:      catalog = ctalog name on Vizier (str)
                join_key = column header to join tables, if multiple (str; optional)

    OUTPUT:     t = data table (AstroPy Table)

    """

    ### GET FULL CATALOG (ALL COLUMNS, ALL ROWS)
    viz = Vizier(catalog=catalog, columns=['**'])
    viz.ROW_LIMIT = -1
    tv = viz.get_catalogs(catalog)

    ### IF MULTIPLE TABLES, JOIN THEN
    for i, val in enumerate(tv.keys()):
        if i == 0:
            t = tv[val]
        else:
            tt = tv[val]
            if join_key in tt.columns:
                t = join(t, tt, join_type='inner', keys=join_key)

    return t

========================== Code ==========================

In [ ]:
### LOAD IN LUPUS DATA
T = get_data("J/ApJ/828/46")
T.sort('RAJ2000')

In [ ]:
### WRITE HEADER INFO
f = open('../output/table_03.tex', 'w')
f.write(r'\capstartfalse'                                  + ' \n')
f.write(r'\begin{deluxetable*}{lrrrrrrr}'                    + ' \n')
f.write(r'\tabletypesize{\footnotesize}'                   + ' \n')
f.write(r'\centering'                                      + ' \n')
f.write(r'\tablewidth{500pt}'                              + ' \n')
f.write(r'\tablecaption{Gas Properties \label{tab-gas}}'   + ' \n')
f.write(r'\tablecolumns{8} '                               + ' \n')
f.write(r'\tablehead{'                                     + ' \n')
f.write(r' \colhead{Source}'                               + ' \n')
f.write(r'&\colhead{$F_{\rm 13CO}$}'                       + ' \n')
f.write(r'&\colhead{$E_{\rm 13CO}$}'                       + ' \n')
f.write(r'&\colhead{$F_{\rm C18O}$}'                       + ' \n')
f.write(r'&\colhead{$E_{\rm C18O}$}'                       + ' \n')
f.write(r'&\colhead{$M_{\rm gas}$}'                        + ' \n')
f.write(r'&\colhead{$M_{\rm gas,min}$}'                    + ' \n')
f.write(r'&\colhead{$M_{\rm gas,max}$} \\'                 + ' \n')
f.write(r' \colhead{}'                                     + ' \n')
f.write(r'&\colhead{(mJy~km~s$^{-1}$)}'                    + ' \n')
f.write(r'&\colhead{(mJy~km~s$^{-1}$)}'                    + ' \n')
f.write(r'&\colhead{(mJy~km~s$^{-1}$)}'                    + ' \n')
f.write(r'&\colhead{(mJy~km~s$^{-1}$)}'                    + ' \n')
f.write(r'&\colhead{($M_{\rm Jup}$)}'                      + ' \n')
f.write(r'&\colhead{($M_{\rm Jup}$)}'                      + ' \n')
f.write(r'&\colhead{($M_{\rm Jup}$)}'                      + ' \n')
f.write(r'}'                                               + ' \n')
f.write(r'\startdata'                                      + ' \n')

In [ ]:
lim = 10
for i, val in enumerate(np.zeros(lim)):

    source = T['Name'][i]

    if T['l_Mgas'][i] != '<':
        mgas = "{0:0.1f}".format(T['Mgas'][i])

    else:
        mgas = r'$' + str(T['l_Mgas'][i]) + "{0:0.1f}".format(T['Mgas'][i]) + r'$'

    if np.ma.is_masked(T['b_Mgas'][i]) is True:
        b_mgas = '...'
    else:
        b_mgas = r'$' + "{0:0.1f}".format(T['b_Mgas'][i]) + r'$'

    if np.ma.is_masked(T['B_Mgas'][i]) is True:
        B_mgas = '...'
    else:
        B_mgas = r'$' + "{0:0.1f}".format(T['B_Mgas'][i]) + r'$'


    if T['l_F13CO'][i] != '<':
        f13 = "{0:01}".format(T['F13CO'][i])
        e13 = "{0:01}".format(T['e_F13CO'][i])
    else:
        f13 = r'$' + str(T['l_F13CO'][i]) + "{0:01}".format(T['F13CO'][i]) + r'$'
        e13 = '...'


    if T['l_F18CO'][i] != '<':
        f18 = "{0:01}".format(T['F18CO'][i])
        e18 = "{0:01}".format(T['e_F18CO'][i])
    else:
        f18 = r'$' + str(T['l_F18CO'][i]) + "{0:01}".format(T['F18CO'][i]) + r'$'
        e18 = '...'

    if (i < lim - 1):
        end = r' \\' + '\n'
    else:
        end = '\n'

    f.write(source + ' & ' + f13 + ' & ' + e13 + ' & ' + f18 + ' & ' + e18 + ' & ' + 
            mgas + ' & ' + b_mgas + ' & ' + B_mgas + end)

In [ ]:
f.write(r'\enddata' + ' \n')
f.write(r'\tablenotetext{}{(This table is available in its entirety in machine-readible form.)}' + ' \n')
f.write(r'\end{deluxetable*}' + ' \n')
f.write(r'\capstartfalse' + ' \n')
f.close()